In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/delhi-house-price-prediction/MagicBricks.csv')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

 # >> **Exploratory Data Analysis**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### So lets start the Data Analysis:

In [ ]:
df['Area'].value_counts()

## To get a clear view about Area, we can see these plots below.
   ###     ----Which clearly says that Area values below 3000 sqft are more frequent.

In [ ]:
sns.regplot(y='Area',x='Price',data=df)

In [ ]:
sns.boxplot(df["Area"])
plt.show()

In [ ]:
sns.barplot(df["Price"],df['Area'])
plt.show()

In [ ]:
sns.scatterplot(df["Area"],df['Price'])
plt.show()

In [ ]:
df['BHK'].value_counts()

In [ ]:
sns.regplot(y='BHK',x='Price',data=df)

In [ ]:
sns.countplot(df["BHK"])
plt.show()

In [ ]:
sns.boxplot(df["BHK"])
plt.show()

### **From these three plots above we can understand that BHK above 5 are not significant to predict Price. Lets do the same for Bathrooms**

In [ ]:
df['Bathroom'].value_counts()

In [ ]:
sns.regplot(y='Bathroom',x='Price',data=df)

In [ ]:
sns.countplot(df["Bathroom"])
plt.show()

### **Now this analysis implies that No. of Bathrooms above 5 really don't have any significant effect on Price Prediction**

In [ ]:
df['Parking'].value_counts()

In [ ]:
sns.boxplot(df["Parking"])
plt.show()

### **No. of Parking is mainly distributed about 1 and 2. This will be discussed later**

## ***For now let us analyse the Catagorical Data:-***

In [ ]:
df['Transaction'].value_counts()

In [ ]:
sns.catplot(x="Transaction", y="Area", data=df)


In [ ]:
sns.catplot(x="Transaction", y="Area", kind="box", data=df)

In [ ]:
sns.catplot(x="Transaction", y="Area", kind="bar", data=df)

### ***Area is the biggest factor to predict price, as we all know. And seemingly Box Plot is a better way to understand the distribution of these Catagorical Data over Area. These plots as well say that MAX of Area is around 2500 sqft.*** 

### **Lets see what the other data are signifying--**

In [ ]:
df['Status'].value_counts()

In [ ]:
sns.catplot(x="Status", y="Area", kind="box", data=df)

In [ ]:
df['Furnishing'].value_counts()

In [ ]:
sns.catplot(x="Furnishing", y="Area", kind="box", data=df)

In [ ]:
df['Type'].value_counts()

In [ ]:
sns.catplot(x="Type", y="Area", kind="box", data=df)

### **These are also making us to take Area values below 3000 sqft. Lets see what the Area itself says**

In [ ]:
df['Locality'].value_counts()

In [ ]:
sns.catplot(x="Area", y="Locality", kind="bar", data=df)

### **Locality Column is a mess actually, 365 different localities are there. Though its evident. So we actually have nothing to do with this column.**

# >>  Filtering

> ## **We have done the Analysis. So now we will filter out the Dataframe. And we will workout the above plans.**

## 1. Filtering out Area Column-->

### Some more codes to get the correct Area range ;)

In [ ]:
area_df=df['Area'].value_counts().head(40)
# Areas that occured atleast 10 times are taken:--

area_df
# among top 36 most occured(atleast 10 times) Area values, MAX is 2700.

In [ ]:
sns.distplot(df['Area'])

In [ ]:
plt.hist(df["Area"],color='green')
plt.show()

 ## **~Yup. We did well. Lets keep AREA<=3000 sqft**

In [ ]:
df=df[df['Area']<=3000]

In [ ]:
sns.regplot(y='Area',x='Price',data=df)

> ### And you can clearly see that with this above plot.

## 2. Filtering out BHK Column-->

In [ ]:
df=df[df['BHK']<6]

## 3. Filtering out Bathroom Column-->

In [ ]:
df=df[~(df['Bathroom']==6)]

### Here's our BRAND-NEW Dataframe....

In [ ]:
df


In [ ]:
correlation=df.corr()

In [ ]:
correlation

### **Actully there are some Duplicate rows in our df. So I'm dropping them.

In [ ]:
df.shape

In [ ]:
new=df.drop_duplicates()

In [ ]:
new.shape

# >> NULL value Removing

In [ ]:
new.isnull().sum()

### There are still some NULL values in our Dataset. Lets find out ways to deal with them. 
#### *First concern is for Per_Sqft.*

In [ ]:
plt.hist(new["Per_Sqft"],color='brown')
plt.show()

### **There are 200 odd NULL values in Per_Sqft. I think the best way to fill them will be to fill them by the ratio of corresponding Price and Area.**

## "And it has a Reason"

In [ ]:
compare=(new['Price']/new['Area'])-new['Per_Sqft']

In [ ]:
compare

In [ ]:
compare.median()

### You can see Per_Sqft value is quite close to that ratio. So we will stick to this plan.

In [ ]:
new['Per_Sqft']=new['Per_Sqft'].fillna(value=new['Price']/new['Area'] )

In [ ]:
new.head(50)

In [ ]:
new.isnull().sum()

In [ ]:
new[new['Bathroom'].isnull()]

In [ ]:
new['Bathroom']=new['Bathroom'].fillna(value=new['Bathroom'].median())

In [ ]:
# Not needed to go with Float value to Bathroom.
new = new.astype({"Bathroom":'int64'}) 

In [ ]:
new

In [ ]:
new.isnull().sum()

In [ ]:
new['Furnishing'].value_counts()

In [ ]:
new['Type'].value_counts()

In [ ]:
new[new['Furnishing'].isnull()]

In [ ]:
new[new['Type'].isnull()]

### Type Column is well distributed between Builder_Floor and Apartment. What should we do?

### Ok. We are filling it with that value of a room at the same locality. Any other Suggestion is always welcomed.

In [ ]:
new=new.sort_values(by='Locality',axis=0)
#This will arrange the localities sequentially...

In [ ]:
new

In [ ]:
new.isnull().sum()

In [ ]:
new['Type']=new['Type'].fillna(method='bfill')

In [ ]:
new['Furnishing']=new['Furnishing'].fillna(method='bfill')

In [ ]:
new.isnull().sum()

### Parking still have some NaNs. Lets first find if it has any significant role to play to predict price or not. 

## For that correlation of various columns with Price(i.e the Output Column) is to be examined.
### But some columns are NOT Numerical.

# >>LABEL ENCODING:
##### >----to convert Furnishing, Status, Transaction and Type into Numerical Data----<

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
furnishing_encoder=LabelEncoder()
status_encoder=LabelEncoder()
transaction_encoder=LabelEncoder()
type_encoder=LabelEncoder()

In [ ]:
new['Furnishing']=furnishing_encoder.fit_transform(new['Furnishing'].astype('str'))
new['Status']=status_encoder.fit_transform(new['Status'])
new['Transaction']=transaction_encoder.fit_transform(new['Transaction'])
new['Type']=type_encoder.fit_transform(new['Type'].astype('str'))

In [ ]:
new.head(30)

In [ ]:
new.corr()['Price']

In [ ]:
sns.heatmap(new.corr(),annot=True)

In [ ]:
sns.regplot(x="Parking", y="Price", data=new)


In [ ]:
sns.catplot(x="Furnishing", y="Price", kind="box", data=new)

In [ ]:
sns.regplot(x="Furnishing", y="Price", data=new)

> ## **Well we can understand from both Correlation Chart and the above plot that Parking Column is Good for Nothing for Price-Prediction. "A correlation of 0.02 is nothing". So is with Furnishing.**
### So these are dropped off along with Locality....

In [ ]:
final= new.drop(columns=['Furnishing','Parking','Locality'])

In [ ]:
final

In [ ]:
final['Bathroom'].value_counts()

### "And we are left with a pure numerical dataset."

# >>ONE HOT ENCODING:

### **As Label Encoding is generally followed by OHE

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(drop='first')

## Columns to be OHEd:~ BHK and Bathroom, as they are like catagorical data. Status, Transaction and Type are not reqd to do so as they have only two columns each. 

In [ ]:
final2=ohe.fit_transform(final[['BHK','Bathroom']])

In [ ]:
final2.toarray()

In [ ]:
final2=pd.DataFrame(final2.toarray())

In [ ]:
final2

# >>DATA Spliting:

In [ ]:
final1=final.drop(columns=['BHK','Bathroom'])

In [ ]:
final1=final1.reset_index()

In [ ]:
final1=final1.drop(columns=['index'])

In [ ]:
final1

In [ ]:
final= pd.concat([final1, final2], axis=1, ignore_index=False)

In [ ]:
final

In [ ]:
y=final['Price'].values

In [ ]:
y

# >>SCALING: 
### --- a step of Data Pre-processing which is applied to independent variables or features of data. 
### *~It basically helps to normalise the data within a particular range. Sometimes, it also helps in speeding up the calculations in an algorithm*.
> ## The formula used in the following object is often used in Statistic, where each value is replaced first substracting the mean of that column and then divided by the Standard Deviation of the same.
https://www.google.com/url?sa=i&url=https%3A%2F%2F365datascience.com%2Fstandardization%2F&psig=AOvVaw3pa_eeGRJHaJSfXrYJCCbF&ust=1606203348694000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCJCbpvOTmO0CFQAAAAAdAAAAABAD

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [ ]:
X=final.drop(columns=['Price'])
X

In [ ]:
X=scaler.fit_transform(X)
X

In [ ]:
print(X.shape)
print(y.shape)

# >> MODEL Building:

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.9,random_state=2)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## **(1). Linear Regression:--**

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
y_pred=lr.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

In [ ]:
plt.scatter(x=y_test, y=y_pred)
plt.show()

In [ ]:
print(lr.coef_)
print(lr.intercept_)

## **(2). Random Forest:--**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor(n_estimators=100,random_state=0)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred1=rf.predict(X_test)

In [ ]:
y_pred1

In [ ]:
r2_score(y_test,y_pred1)

In [ ]:
plt.scatter(x=y_test, y=y_pred1)
plt.show()

In [ ]:
# Try different numbers of n_estimators - this will take a minute or so
estimators = np.arange(10, 200,10)
scores = []
for n in estimators:
    rf.set_params(n_estimators=n)
    rf.fit(X_train, y_train)
    scores.append(rf.score(X_test, y_test))
plt.title("Effect of n_estimators")
plt.xlabel("n_estimator")
plt.ylabel("score")
plt.plot(estimators, scores)

In [ ]:
estimators

In [ ]:
scores

## **(3). KNeighbours:--**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knr=KNeighborsRegressor()

In [ ]:
knr.fit(X_train, y_train)

In [ ]:
y_pred2=knr.predict(X_test)

In [ ]:
y_pred2

In [ ]:
r2_score(y_test,y_pred2)

## **(4). Gradient Boosting:--**

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
GB = GradientBoostingRegressor(loss='ls',n_estimators=800,subsample=0.1,criterion='mse',max_features='auto',min_samples_leaf=5, min_samples_split=4, max_leaf_nodes=40, learning_rate=0.05, max_depth=20, random_state=2, n_iter_no_change=5)

In [ ]:
GB.fit(X_train, y_train)
y_pred3 = GB.predict(X_test)
r2_score(y_test,y_pred3)

In [ ]:
y_pred3

## **(5). XGBoost:--**

In [ ]:
from xgboost import XGBRegressor
XGB = XGBRegressor(eta=0.05, gamma=1,tree_method='auto',objective ='reg:squarederror',max_depth=4, max_leaves=15, subsample=0.1, reg_lambda=9, reg_alpha=50, n_estimators=900, random_state=30)

In [ ]:
XGB.fit(X_train, y_train)
y_pred4 = XGB.predict(X_test)
r2_score(y_test,y_pred4)

# ---> There you go. We have successfully generated a model that is capable of predicting the Price of a house if the following data are given by the user.
### 1.Total Area, 
### 2.BHK,
### 3.Bathroom,
### 4.Status,
### 5.Transaction,
### 6.Type and 
### 7.Per_Sqft value. 


* ## Predictions are more than 81% correct(with the Gradient Boosting Model), which is quite good. XGBoost is unlikely loosing the race to GB, which is kind of weird, as it generally is better. But, we can do further upgradation and make the predictions more accurate possibly with some other Models. 
 ### **But for now the Boosting Algorithms are GOOD to GO.**

# >> Pickling

In [ ]:
import pickle

In [ ]:
pickle.dump(lr, open('model.pkl','wb'), protocol=2)

In [ ]:
pickle.dump(status_encoder, open('status_encoder.pkl','wb'), protocol=2)
pickle.dump(transaction_encoder, open('transaction_encoder.pkl','wb'), protocol=2)
pickle.dump(type_encoder, open('type_encoder.pkl','wb'), protocol=2)
pickle.dump(ohe, open('ohe.pkl','wb'), protocol=2)

**To CSV**

In [ ]:
final.to_csv('House_Price.csv',header=False)

 ## ***I hope this Notebook was helpful for you, and may be you learned something from it. I'm eagerly waiting for your suggestions, so that the results can be further improved. So, comment below your sayings.*** 
 ###  **THANK YOU, Be Safe.....**